### Pipeline 내부 실행

In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!"
    ]
)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

내부 수행 과정

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)

### 토크나이저를 이용한 전처리

파이프라인의 첫번째 단계\
텍스트 입력을 모델이 이해할 수 있는 숫자로 변환하기

토크나이저
* 입력을 토큰으로 분할
* 각 토큰을 정수로 매핑
* 모델에 유용할 수 있는 부가적인 입력 추가

`AutoTokenizer` `from_pretrained()` 를 사용하여 모델이 사전 학습될 때와 동일한 방식으로 수행

In [2]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer 생성하면 토크나이저에 문장을 입력해서 모델에 전달할 수 있는 파이썬 딕셔너리 정보를 구할 수 있다.

이후에 input IDs 리스트를 텐서로 변환해주면 된다

In [3]:
# 텐서의 유형 지정하기
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(
    raw_inputs,
    padding=True,
    truncation=True,
    return_tensors="pt"
)
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


출력은 두개의 키를 가지는 파이썬 딕셔너리이다.
1. `input_ids` 토큰의 고유 식별자로 구성된 두 행의 정수
2. `attention_mask`

### 모델

In [4]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 고차원 벡터

Transformer 모듈이 가지는 일반적인 차원
1. 배치 크기
2. 시퀀스 길이
3. 은닉 크기

Trnasformer 모델의 출력은 `namedtuple` 이나 딕셔너리처럼 동작

접근 방법
1. `outputs["last_hidden_state"]`
2. `outputs[0]`

In [5]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


### 모델 헤드

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg)

Transformers는 다양한 아키텍처가 있고, 각 아키텍처는 특화된 작업을 처리하도록 설계되어 있다

1. Model
2. ForCausalLM
3. ForMaskedLM
...

예시에서는 문장의 긍정/부정을 분류하기 위해 시퀀스 분류 헤드가 포함되어 있는 모델이 필요하다.\
따라서 `AutoModel` 클래스 대신 `AutoModelForSequenceClassification` 을 사용한다.

In [7]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [11]:
# shape 보기
# 두 개의 문장, 두 개의 레이블 -> (2,2) shape
print(outputs.logits.shape)

torch.Size([2, 2])


### 출력 후처리

정규화되지 않은 원시 점수인 __logits__ 을 확률로 변환하는 과정이 필요하다

__SoftMax__ 계층 통과시키기

In [12]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4419e-04]], grad_fn=<SoftmaxBackward0>)


In [13]:
# 해당하는 레이블 가져오기
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}